# X3D (eXpanded 3D) Implementation - Design Alternative 1

`Warning:` Do not run all the code, read the notes first before running them as they may reduce your device's memory when not careful, leading to a kernel crash or Out-Of-Memory (OOM) Error, sige ka magla-lag yang pc mo

`Note`: Isipin mo alternate universe na training toh

**Local System Specifications used to train the model:** <br>
| Component  | Device   |
| :------------- | :--------------- | 
| CPU   | AMD Ryzen 5 5600x     | 
| GPU   | Nvidia GeForce RTX 3070  8GB   | 
| RAM   | 32 GB     | 


`Note`: This notebook's codes will come from [PytorchVideo](https://pytorchvideo.org/docs/tutorial_overview) and from this Talha Anwar's [YouTube Playlist](https://www.youtube.com/playlist?list=PLtGXgNsNHqPRAscIi6dMUuPCfDQ1Vho2U)

#### Import Libraries

In [ ]:
import sys
sys.path.append('pytorchvideo')
import torchvision.transforms.functional as F_t

sys.modules["torchvision.transforms.functional_tensor"] = F_t
import pytorchvideo
from pytorchvideo.data import *
import pytorch_lightning
import torchmetrics
import torch

import pandas as pd
import pathlib
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pathlib.PosixPath = pathlib.WindowsPath # For windows only

#### Prepare the dataset

In [3]:
dataset_dir = pathlib.Path(r"./PD_Dataset")

In [4]:
def renameFile():
    for file_path in dataset_dir.rglob("*.avi"):
        # Get the parent directory name and clean it up
        dir_name_clean = file_path.parent.name.replace(" ", "_")
        # Clean the file name by replacing spaces with underscores
        file_clean = file_path.name.replace(" ", "_")
        
        # Create the new file name
        new_name = f"{dir_name_clean}_{file_clean}"
        new_path = file_path.parent / new_name
        
        print(f"Renaming: {file_path} -> {new_path}")
        file_path.rename(new_path)

In [5]:
#renameFile() #run once...

In [6]:
categories = ["BarbellCurl", "Deadlift", "LateralRaises", "OverheadPress", "Squat"]

video_labels = [
    [video for video in (dataset_dir / category).iterdir() if video.is_file()]
    for category in categories
]

# Flatten and create labels
label = [i for i, videos in enumerate(video_labels) for _ in videos]
combined = [video for sublist in video_labels for video in sublist]

print(label)
print(combined)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [7]:
df = pd.DataFrame(zip(combined, label), columns=["file", "label"])
df.head()

,file,label
0,PD_Dataset\BarbellCurl\BarbellCurl_barbell_cur...,0
1,PD_Dataset\BarbellCurl\BarbellCurl_barbell_cur...,0
2,PD_Dataset\BarbellCurl\BarbellCurl_barbell_cur...,0
3,PD_Dataset\BarbellCurl\BarbellCurl_barbell_cur...,0
4,PD_Dataset\BarbellCurl\BarbellCurl_barbell_cur...,0


In [8]:
df["label"].value_counts()

label
3    157
4    132
2    125
1    124
0    123
Name: count, dtype: int64

data is not balanced so... Remove tayo randomly :)

In [9]:
## Function to balance
def balance_dataset(df):
    # Find the minimum count among all classes
    min_count = df["label"].value_counts().min()
    
    # Downsample each class to match min_count
    balanced_df = (
        df.groupby("label").apply(lambda x: x.sample(n=min_count, random_state=43)).reset_index(drop=True)
    )
    
    return balanced_df

In [10]:
df = balance_dataset(df)
df["label"].value_counts() # Check for the number of data

C:\Users\kuron\AppData\Local\Temp\ipykernel_13420\830260054.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby("label").apply(lambda x: x.sample(n=min_count, random_state=43)).reset_index(drop=True)


label
0    123
1    123
2    123
3    123
4    123
Name: count, dtype: int64

#### Split the dataset

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

train_df, val_df = train_test_split(df, test_size=0.3, shuffle=True)
val_df, test_df = train_test_split(val_df, test_size=0.1, shuffle=False)

In [12]:
len(train_df), len(val_df), len(test_df)

(430, 166, 19)

In [13]:
train_df.to_csv("train_df.csv", header=False , sep=" ", index=False) 
val_df.to_csv("val_df.csv", header=False, sep=" ", index=False)
test_df.to_csv("test_df.csv", header=False, sep=" ", index=False)

#### Prepare and augment the dataset for better generalization of the mudel 

In [ ]:
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize, RandomShortSideScale,
    UniformTemporalSubsample, Permute
)

from torchvision.transforms import (
    Compose, Lambda, RandomCrop,
    RandomHorizontalFlip, Resize
)

from torchvision.transforms._transforms_video import (
    CenterCropVideo, NormalizeVideo
)

In [ ]:
video_transform = Compose([
    ApplyTransformToKey(key='video',
    transform=Compose([
        UniformTemporalSubsample(20),
        Lambda(lambda x: x/255.0),
        Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
        RandomShortSideScale(min_size=112, max_size=128),
        CenterCropVideo(112),
        RandomHorizontalFlip(p=0.5)
    ])
    )
])

In [ ]:
train_dataset = labeled_video_dataset("train_df.csv",
                                      clip_sampler=make_clip_sampler('random', 2),
                                      transform=video_transform,
                                      decode_audio=False
                                      )
loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=
)